<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/PVGIS_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install requests pandas

In [32]:
import pandas as pd
import requests
import io
import time

cities = {
    "Turku": {"lat": 60.45, "lon": 22.26},
    "Copenhagen": {"lat": 55.67, "lon": 12.56},
    "Stockholm": {"lat": 59.32, "lon": 18.06},
    "Oslo": {"lat": 59.91, "lon": 10.75}
}

base_url = "https://re.jrc.ec.europa.eu/api/v5_3/seriescalc"

for city, coords in cities.items():
    params = {
        'lat': coords['lat'],
        'lon': coords['lon'],
        'startyear': 2023,
        'endyear': 2023,
        'outputformat': 'csv'
    }

    try:
        print(f"Downloading data for {city}...")
        response = requests.get(base_url, params=params)

        if response.status_code != 200:
            print(f"❌ Server error for {city}. Code: {response.status_code}")
            continue

        lines = response.text.splitlines()
        header_index = next((i for i, line in enumerate(lines) if line.startswith("time,")), -1)

        if header_index == -1:
            print(f"❌ Data header not found for {city}.")
            continue

        # Extract only the 8760 hours of the year + the header
        # This ignores the legend text at the bottom that was causing the error
        data_lines = lines[header_index : header_index + 8761]

        df = pd.read_csv(io.StringIO("\n".join(data_lines)))

        # --- COLUMN DETECTION ---
        # Based on your error, the column is 'G(i)'
        ghi_col = [c for c in df.columns if c.startswith('G(')][0]
        temp_col = 'T2m'
        wind_col = 'WS10m'

        # 1. Format Time (Now safe from footer text)
        df['dt_obj'] = pd.to_datetime(df['time'], format='%Y%m%d:%H%M')
        df['Date'] = df['dt_obj'].dt.date
        df['Hour'] = df['dt_obj'].dt.strftime('%H:%M')

        # 2. Select and Rename
        df_filtered = df[['Date', 'Hour', ghi_col, wind_col, temp_col]].copy()
        df_filtered.columns = ['Date', 'Hour', 'GHI (W/m2)', 'Wind Speed (m/s)', 'Air Temperature (C)']

        # 3. Save
        filename = f"{city.lower()}_2023_data.csv"
        df_filtered.to_csv(filename, index=False)
        print(f"✅ Successfully saved: {filename}")

        time.sleep(2)

    except Exception as e:
        print(f"❌ Error processing {city}: {e}")

print("\nProcess finished. You should now have 4 separate files.")

✅ Successfully saved: turku_2023_data.csv
✅ Successfully saved: copenhagen_2023_data.csv
✅ Successfully saved: stockholm_2023_data.csv
✅ Successfully saved: oslo_2023_data.csv

Process finished. You should now have 4 separate files.
